In [ ]:
!pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-uw7vl8wt
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-uw7vl8wt
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 44.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.6 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567358 sha256=f95ba522302a34bbe91a2212c970daf22404cc275d49a9f2977e95418cf778d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-vmo

In [ ]:
!nvidia-smi

Sun Sep  1 10:41:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import nltk
import os

# Set NLTK data path to the directory within the virtual environment
nltk_data_dir = os.path.join(os.getcwd(), '.venv', 'nltk_data')
nltk.data.path.append(nltk_data_dir)

# Download the 'punkt' tokenizer to this directory
nltk.download('punkt', download_dir=nltk_data_dir)
nltk.download('punkt_tab', download_dir=nltk_data_dir)


[nltk_data] Downloading package punkt to /content/.venv/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /content/.venv/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import nltk
import os

from bark import generate_audio, preload_models, SAMPLE_RATE
import time
from scipy.io.wavfile import write as write_wav

#import naming_saving as ns
#import split

In [ ]:

from scipy.io.wavfile import write as write_wav

def __get_unique_filename(base_name, directory):
    """Generate a unique filename by appending an incrementing number."""
    # Check if the directory exists
    if not os.path.isdir(directory):
        raise ValueError(f"The directory {directory} does not exist.")

    # Get a list of existing filenames in the directory
    existing_files = os.listdir(directory)

    # Extract existing numbers from filenames
    existing_numbers = []
    for filename in existing_files:
        if filename.startswith(base_name) and filename[len(base_name)].isdigit():
            try:
                number = int(filename[len(base_name):].split('.')[0])
                existing_numbers.append(number)
            except ValueError:
                continue

    # Find the highest number and increment
    highest_number = max(existing_numbers, default=0)
    next_number = highest_number + 1

    unique_filename = f"{base_name}{next_number}"

    return unique_filename

def save_text_file_with_unique_name(content, base_name, directory, extension='txt'):
    """Save a text file with a unique name in the specified directory."""
    # Generate a unique filename
    filename = __get_unique_filename(base_name, directory) + f".{extension}"

    # Full path to save the file
    filepath = os.path.join(directory, filename)

    # Save the file
    with open(filepath, 'w') as file:
        file.write(content)

    return filepath

def save_wav_file_with_unique_name(audio_array, base_name, directory, sample_rate):
    """Save a wav file with a unique name in the specified directory."""

    # Generate a unique filename
    filename = __get_unique_filename(base_name, directory) + ".wav"

    # Full path to save the file
    filepath = os.path.join(directory, filename)

    write_wav(filepath, sample_rate, audio_array)

    return filepath

In [ ]:
import re
import nltk

# run 'python download_nltk.py' first

def split_text_to_sentence_chunks(text_or_file, min_words=5, max_words=9):
    if hasattr(text_or_file, 'read'):
        # If it's a file object, read the text
        text = text_or_file.read()
    else:
        # If it's not a file object, assume it's a string
        text = text_or_file

    # Tokenize text into sentences
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_word_count = 0

    for sentence in sentences:
        sentence_words = nltk.word_tokenize(sentence)
        sentence_word_count = len(sentence_words)

        if current_word_count + sentence_word_count > max_words:
            if current_word_count >= min_words:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
                current_word_count = sentence_word_count
            else:
                current_chunk += " " + sentence
                current_word_count += sentence_word_count
        else:
            current_chunk += " " + sentence
            current_word_count += sentence_word_count

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


In [ ]:
history_promtps_pt = [
    "v2/pt_speaker_0",
    "v2/pt_speaker_1",
    "v2/pt_speaker_2",
    "v2/pt_speaker_3",
    "v2/pt_speaker_4",
    "v2/pt_speaker_5",
    "v2/pt_speaker_6",
    "v2/pt_speaker_7",
    "v2/pt_speaker_8",
    "v2/pt_speaker_9"
]

history_promtps_en =[
    "v2/en_speaker_0",
    "v2/en_speaker_1",
    "v2/en_speaker_2",
    "v2/en_speaker_3",
    "v2/en_speaker_4",
    "v2/en_speaker_5",
    "v2/en_speaker_6",
    "v2/en_speaker_7",
    "v2/en_speaker_8",
    "v2/en_speaker_9",
]

In [ ]:
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["SUNO_USE_SMALL_MODELS"] = "False"
#os.environ["SUNO_OFFLOAD_CPU"] = "False"

preload_models()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/bark/generation.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 185MB/s]


In [ ]:
import numpy as np
import soundfile as sf
import os

file_path = "sd.md"
base_output_name = "sd2"
output_directory = "outputs"
output_file_name = f"{base_output_name}.wav"
output_file_path = os.path.join(output_directory, output_file_name)

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Open the output file in 'write' mode
with sf.SoundFile(output_file_path, mode='w', samplerate=SAMPLE_RATE, channels=1) as file:
    with open(file_path, 'r') as text_file:
        chunks = split_text_to_sentence_chunks(text_file)

        t0 = time.time()
        total_duration = 0
        for idx, chunk in enumerate(chunks):
            print(f"Chunk {idx + 1}: {chunk}")
            audio_array = generate_audio(chunk, history_prompt=history_promtps_en[0])
            audio_duration_s = len(audio_array) / SAMPLE_RATE
            total_duration += audio_duration_s
            print(f"Generated {audio_duration_s:.2f}s of audio")

            # Write the audio data to the file
            file.write(audio_array)

        generation_duration_s = time.time() - t0

    print(f"Took {generation_duration_s:.2f}s to generate {total_duration:.2f}s of audio")

/usr/local/lib/python3.10/dist-packages/bark/generation.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with InferenceContext(), torch.inference_mode(), torch.no_grad(), autocast():


Chunk 1: Précis of Cognitive Gadgets: The Cultural Evolution of Thinking

Abstract
Cognitive gadgets are distinctively human cognitive mechanisms – such as imitation, mind
reading, and language – that have been shaped by cultural rather than genetic evolution.


100%|██████████| 36/36 [00:38<00:00,  1.06s/it]
/usr/local/lib/python3.10/dist-packages/bark/generation.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with InferenceContext(), torch.inference_mode(), torch.no_grad(), autocast():


Generated 14.32s of audio
Chunk 2: New gadgets emerge, not by genetic mutation, but by innovations in cognitive development;
they are specialised cognitive mechanisms built by general cognitive mechanisms using
information from the sociocultural environment.


100%|██████████| 35/35 [00:38<00:00,  1.09s/it]


...

Generated 13.95s of audio
Chunk 8: In common with computational approaches (deep learning, predictive
coding, hierarchical reinforcement learning, causal modelling), it emphasises the power of
general-purpose mechanisms of learning.


100%|██████████| 30/30 [00:32<00:00,  1.10s/it]


Generated 11.85s of audio
Chunk 9: Cultural evolutionary psychology, however, also
challenges use of the behavioural gambit in economics and behavioural ecology, and rejects
the view that human minds are composed of “innate modules” or “cognitive instincts.”


100%|██████████| 33/33 [00:36<00:00,  1.09s/it]


Generated 13.09s of audio
Took 454.80s to generate 117.17s of audio
